In [8]:
# -*- coding: dkpanda, cbenjamin-*-
#4dyncoin-kalinga-correct.ipynb
# SPE vs disorder: created ##

from numpy import *
#from matplotlib.pyplot import *
import numpy as np
import math
import random
from scipy import integrate
#import time
import matplotlib.pyplot as mp

#R=5 #max jump length
#lambda_val = 0.5 #mean of disoder distributions  ---HERE
#0.0 to 1 #disorder strength
#SHHH = np.zeros(N+1)   #...results to not be stored in this array
simulations=500#
N =25 
P=4
# Pre-defs :
pi=np.pi
cos=np.cos
sin=np.sin
sqrt=np.sqrt
eye=np.eye
roll=np.roll
kron=np.kron
zeros=np.zeros
exp=np.exp
empty=np.empty
outer=np.outer
#sqrt=np.sqrt
phi=pi/2
#l=0 #pi/2 # 0
#Needed stuffs :
coin0 = np.array([1, 0]) # |0>
coin1 = np.array([0, 1]) # |1>
C00 = np.outer(coin0, coin0) # |0><0|
C01 = np.outer(coin0, coin1) # |0><1|
C10 = np.outer(coin1, coin0) # |1><0|
C11 = np.outer(coin1, coin1) # |1><1|

C_hat = (cos(pi/4)*C00 + sin(pi/4)*C01 + sin(pi/4)*C10 - cos(pi/4)*C11) #
K_hat = (C01 + C10) #Grover coin
F_hat = (C00 + 1j*C01 + 1j*C10 + C11)/sqrt(2.) #
M_hat = (1j*C00 + C01 - C10 - 1j*C11)/sqrt(2.) #
Id_mat=C00+C11 #
Y_hat = (-1j*C01 + 1j*C10) #
#NEW
def coin(rho, alpha, beta):
  gcoin = sqrt(rho)*C00 + sqrt(1-rho)*exp(1j*alpha)*C01 + sqrt(1-rho)*exp(1j*beta)*C10 -sqrt(rho)*exp(1j*(alpha+beta))*C11
  return gcoin
C1=coin(1/2,pi/3,2*pi/3) #P2
C2=coin(1/2,pi/3,pi/6)
prod=C1.dot(C1)
#print(prod)
#C1=coin((1-cos(2*2*pi/9))*(2/3),0,0)
#A1=coin(0.264734,0,0)
#B1=coin(0.801571,0,0)
A1=coin(0.0948578,0,0)
B1=coin(0.827385,0,0)
def dcoin(w,xi):
  r=(1+w*xi)/2
  gcoin = sqrt(r)*C00 + sqrt(1-r)*C01 + sqrt(1-r)*C10 -sqrt(r)*C11
  return gcoin

#----------------------------------------
def Schmidt(m,l, phi,P,S_hat,H,X,F,M,randcoin): #Calc utpo 25
        posn0 = zeros(P)
        posn0[0] = 1
        #l = random.randint(0, 180)
        #phi = random.randint(0, 360)
        psi0 = kron(posn0,(cos(l/2)*coin0 + exp(1j*phi)*sin(l/2)*coin1))
        psiN = psi0 # Initialisation
        for i in range(1, m + 1):
            D_hat=dcoin(0,randcoin[i-1])       
            D=S_hat.dot(kron(eye(P), D_hat))
            psiN = np.linalg.matrix_power(D,1).dot(psiN)
            #if i%3==0:
                #psiN = np.linalg.matrix_power(H,1).dot(psiN)
            #else:
                #psiN = np.linalg.matrix_power(H,1).dot(psiN)
        prob1 = empty(P)
        prob2 = empty(P)
        prob3 = empty(P)
        prob = empty(P)
        for k in range(P):
            posn = zeros(P)
            posn[k] = 1
            M_hat_k = kron(outer(posn,posn), eye(2))
            X_hat_k = kron(outer(posn,posn), C00)
            W_hat_k = kron(outer(posn,posn), C11)
            proj = M_hat_k.dot(psiN)
            proj1 = X_hat_k.dot(psiN)
            proj01 = X_hat_k.dot(kron(posn,(coin0)))
            proj2 = W_hat_k.dot(psiN)
            proj02 = W_hat_k.dot(kron(posn,(coin1)))
            prob1[k] = (proj02.dot(proj2.conjugate())*proj01.dot(proj1.conjugate()).conjugate()).real
            prob2[k] = (proj02.dot(proj2.conjugate())*proj01.dot(proj1.conjugate()).conjugate()).imag
            prob3[k] = ((proj1.dot(proj1.conjugate())).real - (proj2.dot(proj2.conjugate())).real)/2
        n1 = n2 = n3 = 0
        for j in range(P):
            n1 = prob1[j] + n1
            n2 = prob2[j] + n2
            n3 = prob3[j] + n3
        n = sqrt(n1**2 + n2**2 + n3**2)
        #S = sqrt(0.5 - round(n,14)) + sqrt(0.5 + round(n,14)) #sqrt(0.5 - n) + sqrt(0.5 + n) #
        #Sp = S/((sqrt(2))*pi)
        E1=(0.5 - round(n,13))
        E2=(0.5 + round(n,13))
        #print(E1)
        if E1==0 :
         VEnt = -E2*math.log2(E2)
        else:
         VEnt = -E1*math.log2(E1)-E2*math.log2(E2)
        VEntropy= VEnt/pi
        return  VEntropy #Sp

#...Evaluation...:
      #Integration for the average over all the possible states (all range of theta i.e., l) with phi = pi/2 :

avg_ee=np.zeros(N+1)
for simu in range(simulations):
  #random_jumps = np.random.poisson(lambda_val,N)
  #while any(random_jumps > R) :
    #random_jumps = np.random.poisson(lambda_val , N)
  randcoin=np.random.uniform(-1,1,N)  
  SHHH = np.zeros(N+1)
  for m in range(1, N + 1):
    #P=4
    #jump1=random_jumps[m-1]
    jump=1
    #print("step is and jump length is)
    k1=P #
    out1=outer(zeros(k1),zeros(k1))
    for i1 in range(0,k1):
      out1=out1+outer(eye(1,k1,np.mod(i1-jump,k1)),eye(1,k1,i1)) #
    out2=outer(zeros(k1),zeros(k1))
    for i2 in range(0,k1):
      out2=out2+outer(eye(1,k1,np.mod(i2+jump,k1)),eye(1,k1,i2))  #
    S_hat = kron(out2, C11) + kron(out1, C00)
    H = S_hat.dot(kron(eye(P), C_hat)) #
    X = S_hat.dot(kron(eye(P), K_hat)) #
    F = S_hat.dot(kron(eye(P), F_hat)) #
    M = S_hat.dot(kron(eye(P), M_hat)) #
    A = S_hat.dot(kron(eye(P), A1))  #coin A as in Phys.Rev.A
    B = S_hat.dot(kron(eye(P), B1))  #coin B as in Phys.Rev.A
    C = S_hat.dot(kron(eye(P), C1))  #coin C as in Phys.Rev.A
    #DC=S_hat.dot(dcoinn) #Disorder in coin, different sites will have different coins as per disorder distributions
    Id = S_hat.dot(kron(eye(P), Id_mat)) #
    Y = S_hat.dot(kron(eye(P), Y_hat)) #
        #Now to integrate over theta(l) and phi:
    #dine= Schmidt(m,l, phi,P,ShiftPlus,ShiftMinus,S_hat,H,X,F,M)
    ans, err = integrate.quad(lambda l: Schmidt(m,l, phi,P,S_hat,H,X,F,M,randcoin), 0, pi)
    SHHH[m] = ans
  #print("Simulation no:",simu,"the <E.Entropy>:",SHHH) #print S-normre
  avg_ee=avg_ee+SHHH
  print("Sim: ", simu, "The Corresp. Ensemble averaged EE: ",avg_ee/(simu+1))
avg_ee=avg_ee/simulations

#Save in .dat file(the python output) and show fig:
X=np.arange(0,N+1)
#print(X)
print("Final <E>: \n",avg_ee)
#thanks...

KeyboardInterrupt: 